In [8]:
%pylab inline
from scipy.interpolate import interp1d
import scipy as sc

Populating the interactive namespace from numpy and matplotlib


In [13]:
e = 1  # onsite energy
t = 1 # hopping
U = 1 # coulob repulsion
ne = 1.4 # number of electrons per unit cell/ occupation number
T = 1 # tempreture
n1 = 1 #occupation on site 1
n2 = 1 #occupation on site 1
sigma=0.02

In [10]:
def hamiltonian(U,n1,n2,kxran=linspace(-pi,pi,200,endpoint=False),
                        kyran=linspace(-pi,pi,200,endpoint=False)):
    eigenval=[]
    eigenvec=[]
    for kx in kxran:
        for ky in kyran:
            gamma = 1+exp(-1j*2*kx)+exp(-1j*(kx+ky))+exp(-1j*(kx-ky))
            H = array([[U*n1           ,-1*gamma],
                       [-1*conj(gamma),U*n2     ]])
            w,v =eigh(H)
            eigenval.append(w)
            eigenvec.append(v)
    return array(eigenval),array(eigenvec)

In [11]:
def densty(E,sigma=0.02):
    return sum(1/(sigma*sqrt(2*pi))*exp(-((E-hamiltonian(U,n1,n2)[0].flatten())/sigma)**2/2))

In [17]:
def get_chemical_pot(ne): # ne: number of electrons per unit cell
    NE,eran,_=hist(hamiltonian(U,n1,n2)[0].flatten(),linspace(-6,6,200),cumulative=True,density=True)
    mu= interp1d(NE,eran[:-1])
    return mu(ne)

In [6]:
def num_of_electron_MKII(mu,site,kbt=0.01,use_Fermi=False):
    if use_Fermi:
        avg_num_of_elec = (\
        sum(abs(vects[:,site,0])**2*fermi(vals[:,0]-mu,kbt=kbt))+\
        sum(abs(vects[:,site,1])**2*fermi(vals[:,1]-mu,kbt=kbt)))/num_of_eigen_val
    else:
        avg_num_of_elec = \
        (sum(abs(vects[vals[:,0]<mu,site,0])**2)+\
         sum(abs(vects[vals[:,1]<mu,site,1])**2))/num_of_eigen_val # len hamiltonian 0]
    return avg_num_of_elec

In [ ]:
occin = n1+n2 # occupation in unit cell
occout == 1


n1_up,n2_up = ne/2+delta,ne/2-delta
#n1_do,n2_do = ne/2-delta,ne/2+delta #AFM
n1_do,n2_do = ne/2+delta,ne/2-delta  #FM
while discrepancy >= 10**-5:
    vals_up,vects_up = hamiltonian(U,n1_do,n2_do)
    vals_do,vects_do = hamiltonian(U,n1_up,n2_up)
    # FIX THE STUF DOWN FROM HERE
    num_of_eigen_val = len(vals)
    sum(1/(sigma*sqrt(2*pi))*exp(-((E-vals.flatten())/sigma)**2/2))
    NE,eran,_=hist(vals.flatten(),linspace(-6,6,200),cumulative=True,density=True)
    mu = interp1d(NE,eran[:-1])
    avg_num_of_elec = \
    (sum(abs(vects[vals[:,0]<mu(ne),site,0])**2)+\
    sum(abs(vects[vals[:,1]<mu(ne),site,1])**2))/num_of_eigen_val 

In [ ]:
#class HubbardModel():
    
 #   def __init__(self,on_site_energy,coulomb_repulsion,number_of_electrons_per_unit_cell):
        
  #      self.on_site_energy=on_site_energy
   #     self.coulomb_repulsion=coulomb_repulsion
    #    self.number_of_electrons_per_unit_cell
    